# Analiza funkcji `cacl_base_seq_quality`

W tym notebooku przetestujemy funkcję `cacl_base_seq_quality`, która analizuje jakość sekwencji z pliku FASTQ.

- Porównamy zgodność wyników przy różnych typach wejścia (ścieżka do pliku vs DataFrame).
- Zweryfikujemy poprawność obliczanych statystyk względem oczekiwanych wartości.
- Zmierzymy czas działania funkcji.
- Porównamy wydajność z `fastq-rs`

In [ ]:
import polars_bio as pb
import polars as pl
import pytest
import time

# Ścieżka do przykładowego pliku FASTQ
path = r"C:\Users\Ktos\Desktop\NotatkiPW\SEMESTRII\TBD\PROJEKT\PROJEKT_2\polars-bio-z7\benchmark\src\min_example.fastq"

## Test 1: Sprawdzenie zgodności wyników funkcji przy różnych typach wejścia

Funkcja `cacl_base_seq_quality` może przyjmować:
- ścieżkę do pliku FASTQ
- `DataFrame` utworzony z tego pliku

Sprawdzamy, czy wyniki są identyczne.

In [ ]:
fastq_df = pb.read_fastq(path)

result_from_path = pb.cacl_base_seq_quality(path, output_type="polars.DataFrame")
result_from_df = pb.cacl_base_seq_quality(fastq_df, output_type="polars.DataFrame")

assert result_from_path.frame_equal(result_from_df), "Wyniki różnią się przy różnych typach wejścia"
print(" Test zgodności wyników: OK")

## Test 2: Porównanie wyników z oczekiwanymi wartościami

Dla znanych danych FASTQ sprawdzamy czy wartości statystyk zgadzają się z oczekiwaniami.

In [ ]:
EXPECTED_ROWS = [
    {"position": 0, "min_score": 2.0, "max_score": 31.0, "median_score": 2.0, "q1_score": 2.0, "q3_score": 31.0, "sample_count": 3},
    {"position": 1, "min_score": 16.0, "max_score": 31.0, "median_score": 19.0, "q1_score": 16.0, "q3_score": 31.0, "sample_count": 3},
    {"position": 2, "min_score": 28.0, "max_score": 31.0, "median_score": 31.0, "q1_score": 28.0, "q3_score": 33.0, "sample_count": 3},
]

result = pb.cacl_base_seq_quality(path, output_type="polars.DataFrame")

for i, expected in enumerate(EXPECTED_ROWS):
    row = result[i]
    for key, expected_value in expected.items():
        actual_value = row[key]
        assert actual_value == expected_value, f"Błąd w {key} dla pozycji {i}: oczekiwano {expected_value}, otrzymano {actual_value}"
print(" Test poprawności wartości: OK")

## Pomiar czasu działania funkcji

In [ ]:
start = time.time()
pb.cacl_base_seq_quality(path, output_type="polars.DataFrame")
end = time.time()
print(f" Czas działania funkcji polars_bio: {end - start:.6f} s")

## Porównanie z `fastq-rs`

Jeśli biblioteka `fastq-rs` jest dostępna, możemy porównać wydajność.

In [ ]:
try:
    start = time.time()
    frs_calc_quality(path)
    end = time.time()
    print(f" Czas działania funkcji fastq-rs: {end - start:.6f} s")
except Exception as e:
    print(" Nie można wykonać fastq-rs. Błąd:", e)